In [ ]:
%load_ext sparkmagic.magics

from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()
dsxhi_util.list_livy_endpoints()

In [ ]:
%%spark config
{"executorCores": 4, "numExecutors": 5, "executorMemory": "10g", 
 "driverMemory": "3g", "proxyUser": "jchen-", "driverCores": 1, 
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}

In [ ]:
%spark add -s movermodel -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

In [ ]:
%%spark

print(spark.version)

In [ ]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit

from os.path import expanduser, join, abspath
import time
import pandas as pd

from pyspark.sql.functions import current_date
from pyspark.sql import Window

spark = SparkSession.builder.getOrCreate()

In [ ]:
%%spark

csv_path = '/dev/projects/retention_models/csv_data/'
monthly_path = '/dev/projects/retention_models/monthly_snapshot/'

temp_path = '/dev/projects/aliu_projects/Mover_DL_Temp/'

# Servicing

In [ ]:
%%spark

## Pull servicing book

def servicing(snapshot_dt):
    
    spark.sql('use purchaseretention_servicing_raw_access')

# SRC.ServicingMSP.DLQFILE_master = servicingmsp_dlqfile_master_vw
# SRC.ServicingMSP.USRFILE_master ?
# SRCServicing.GoldenSource.CoreMetrics = srcservicing_goldensource_coremetrics

    df = spark.sql('''  
    SELECT
        last_Day(add_months(current_date(), -1)) as ServiceCalendarDate,
        GS.LN_NO,
        GS.leftportfoliodate,
        GS.leftportfolioreason,
        GS.investornameshort,
        GS.loanphase,
        COALESCE(Orig.OG_NOTE_DT, Orig.OG_CLOSING_DT) as OR_NOTE_DT, 
        LON.begin_effective_date,
        LON.end_effective_date,
        CASE
            WHEN LON.LN_LO_TY IN ('1', '5') THEN 'FHA'
            WHEN LON.LN_LO_TY ='2'          THEN 'VA'
            WHEN LON.LN_LO_TY ='9'          THEN 'USDA'
            WHEN LON.LN_LO_TY ='3'          THEN 'Conventional w/o PMI'
            WHEN LON.LN_LO_TY ='6'          THEN 'Conventional with PMI'
            WHEN LON.LN_LO_TY ='4'          THEN 'Commercial'
            ELSE 'Unknown' END as Loantypedescription,
        LON.LN_1ST_PRIN_BA + (case when PGB.PG_2ND_PRIN_BA is null then 0 else PGB.PG_2ND_PRIN_BA end) as UnpaidBalFirstSecond,
        CASE WHEN LON.LN_ARM_CD = 'Y' THEN 'ARM'
             WHEN LON.LN_ARM_CD = 'N' then 'Fixed' END as LoanAmortizationType,
        ORIG.OG_MTG_AM,
        ORIG.LN_PURPOSE_CD,
        CASE orig.LN_PURPOSE_CD
                when '1' then 'PURCHASE'
                when '2' then 'NOT USED'
                when '3' then 'CONSTRUCTION/PERMANENT'
                when '4' then 'CONSTRUCTION'
                when '5' then 'REFINANCE/PROPERTY IMPROVEMENT'
                when '6' then 'REFINANCE/EQUITY TAKEOUT'
                when '7' then 'REHABILITATION/REMODEL'
                when '8' then 'REFINANCE/NO CASH OUT'
                when '9' then 'OTHER'
                when 'A' then 'HOME IMPROVEMENT'
                when 'B' then 'EDUCATION'
                when 'C' then 'PERSONAL AUTOMOBILE'
                when 'D' then 'MOBILE HOME'
                when 'E' then 'CREDIT CARD'
                when 'F' then 'RECREATIONAL VEHICLE'
                when 'G' then 'OVERDRAFT PROTECTION'
                when 'H' then 'REVOLVING CREDIT'
            End loan_purpose_typeWebBehaviour,
        CASE WHEN Orig.LN_PURPOSE_CD=8 THEN 'Refinance' WHEN Orig.LN_PURPOSE_CD=6 THEN 'Cashout Refi'
             WHEN Orig.LN_PURPOSE_CD=1 THEN 'Purchase' ELSE 'other' END as Ln_Purpose_Type,
        ORIG.OG_OCCUPY_STAT_CD,
        CASE WHEN Orig.OG_OCCUPY_STAT_CD =1 THEN 'Primary Residence'
             WHEN Orig.OG_OCCUPY_STAT_CD =3 THEN 'Investment Property'
             WHEN Orig.OG_OCCUPY_STAT_CD =2 THEN 'Second Home'
             ELSE 'other' END ORIG_OCCUPY_STAT_TYPE,
        LON.LN_ANN_INT_RT,
        LON.LN_TR,
        LON.LN_MONTHLY_PMT_AM,
        LON.LN_ELOC_CD,
        CASE WHEN cast(LON.LN_CRE_QUL_ORIG_CD as double) is not null THEN LON.LN_CRE_QUL_ORIG_CD 
             ELSE LON.LN_CRE_QUL_ORIG_CD END as ORIG_FICO,
        LON.ORIG_LTV_RATIO_FC,
        PTY.PR_VALUE_AM,
                  --PTY.PR_ZIP_CD,
                  --PTY.PR_FIPS_STATE_CD,
                  --PTY.PR_CITY_2_NM,
                  --PTY.PR_FIPS_CNTY_CD,
                  --PTY.PR_ALPHA_STATE_CD,
                  --CASE WHEN LENGTH(PTY.PR_FIPS_STATE_CD) > 1 AND LENGTH(PTY.PR_FIPS_CNTY_CD) >1 THEN CONCAT(PTY.PR_FIPS_STATE_CD,PTY.PR_FIPS_CNTY_CD) ELSE  NULL END as FIPS,
        CASE WHEN ((LON.IV_Id LIKE 'G%'  OR LON.IV_ID LIKE '3G%'))  THEN 'GNMA'
             WHEN  (LON.IV_Id LIKE 'F%'  OR LON.IV_ID IN ('301'))   THEN 'FNMA'
             WHEN  (LON.IV_Id LIKE 'H%'  OR LON.IV_ID like '3H%' )  THEN 'FHLMC'
             WHEN  (LON.IV_Id LIKE '360' OR LON.IV_ID IN ('302' ))  THEN 'Schwab'
             WHEN  LON.IV_Id LIKE '18%'                             THEN 'Quicken - Claims'
             WHEN  LON.IV_Id LIKE '0%'                              THEN 'Quicken Warehouse'
             WHEN  (LON.IV_Id LIKE 'R%' OR LON.IV_Id IN ('150'))    THEN 'Quicken'
             ELSE 'Private Investor'  END as InvestorNameShort_lon ,
        CASE WHEN BOR.BO_NO_OF_BORR_QT > 1 THEN 0 ELSE 1 END as IsSingleBorrower,
        DLQ.co_pay_history,
        DLQ.times_delq_qt
    FROM 
     from servicing_raw_access.goldensource_coremetrics_vw GS
     -- from servicing_src_ods.goldensource_coremetrics GS
    LEFT JOIN 
    (
        SELECT * FROM purchaseretention_servicing_raw_access.servicingmsp_lonfile_master_vw 
        WHERE DELTA_FILE_BYTE <>'D' AND '{0}' BETWEEN to_date(begin_effective_date) AND to_date(end_effective_date)
    ) as LON
    ON GS.ln_no = LON.ln_no and length(LON.ln_no) > 0
    LEFT JOIN 
    (
        SELECT * FROM purchaseretention_servicing_raw_access.servicingmsp_orlfile_master_vw 
        WHERE DELTA_FILE_BYTE <>'D' AND '{0}' BETWEEN to_date(begin_effective_date) AND to_date(end_effective_date)
    ) as ORIG 
    ON GS.ln_no = ORIG.ln_no and length(ORIG.ln_no) > 0
    -- ON LON.ln_no = ORIG.ln_no and length(ORIG.ln_no) > 0
    LEFT JOIN 
    (
        SELECT * FROM purchaseretention_servicing_raw_access.servicingmsp_ptyfile_master_vw 
        WHERE DELTA_FILE_BYTE <>'D' AND '{0}' BETWEEN to_date(begin_effective_date) AND to_date(end_effective_date)
    ) as PTY 
    ON PTY.ln_no = GS.ln_no and length(PTY.ln_no) > 0
    -- ON PTY.ln_no = LON.ln_no and length(PTY.ln_no) > 0
    LEFT JOIN 
    (
        SELECT * FROM purchaseretention_servicing_raw_access.servicingmsp_pgbfile_master_vw 
        WHERE DELTA_FILE_BYTE <>'D' AND '{0}' BETWEEN to_date(begin_effective_date) AND to_date(end_effective_date)
    ) as PGB 
    ON PGB.ln_no = GS.ln_no and length(PGB.ln_no) > 0
    -- ON PGB.ln_no = LON.ln_no and length(PGB.ln_no) > 0
    LEFT JOIN 
    (
        SELECT * FROM purchaseretention_servicing_raw_access.servicingmsp_borfile_master_vw 
        WHERE DELTA_FILE_BYTE <>'D' AND '{0}' BETWEEN to_date(begin_effective_date) AND to_date(end_effective_date)
    ) as BOR
    ON BOR.ln_no = GS.ln_no and length(BOR.ln_no) > 0
    -- ON BOR.ln_no = LON.ln_no and length(BOR.ln_no) > 0
    LEFT JOIN  
    (
        SELECT LoanNumber, max(YearsAtAddress) as liveyears 
        FROM purchaseretention_servicing_raw_access.clientcurrentaddresschangedevent 
        WHERE YearsAtAddress>0 GROUP BY LoanNumber
    ) as ly 
    ON ly.loannumber = GS.ln_no AND length(ly.loannumber) > 0
    -- ON ly.loannumber = LON.ln_no and length(ly.loannumber) > 0
    LEFT JOIN 
    (
        SELECT * FROM purchaseretention_servicing_raw_access.servicingmsp_dlqfile_master_vw 
        WHERE DELTA_FILE_BYTE <>'D' AND '{0}' BETWEEN to_date(begin_effective_date) AND to_date(end_effective_date)
    ) as DLQ
    ON DLQ.ln_no = GS.ln_no and length(DLQ.ln_no) > 0
    -- ON DLQ.ln_no = LON.ln_no and length(DLQ.ln_no) > 0
    WHERE LON.DELTA_FILE_BYTE <>'D'
    AND '{0}' BETWEEN to_date(GS.begineffectivedate) AND to_date(GS.endeffectivedate)
    AND GS.leftportfoliodate is null    '''.format(snapshot_dt))
    
    return df


In [ ]:
%%spark

df_servicing = servicing(snapshot_dt = '2020-07-31')
df_servicing.count()

# ClientIQ

In [ ]:
%%spark

## conformed.clientiq_vw only includes the current information
## sandbox_leadgen.clientiq_hisorical includes all historical datekey in ClientIQ

def client_iq(datekey):
    
    spark.sql('use conformed')
    
    df = spark.sql("""
        SELECT distinct
            loannumber
        ,   loanpurpose as IQ_loanpurpose
        ,   isqlms
        ,   iscurrentqlclient
        ,   sourcereferenceid -- GCID
        ,   experian_investmentproperty
        ,   clientincomeannualamount
        ,   isinvestmentproperty
        ,   annincome
        ,   luid
        ,   isprimaryborrower
        ,   borrowerstatus
        ,   finalcreditscore
        ,   omniview_individual_id
        ,   datekey
        FROM conformed.clientiq_vw
        WHERE iscurrentqlclient = 'TRUE'
            AND datekey = '{0}'
    """.format(datekey))
    
    return df

In [ ]:
%%spark

df_clientIQ = client_iq(datekey = '2020-08-20')

# Experian

In [ ]:
%%spark

def load_ExperianData_id(match_dt, experian_dt):
    
    spark.sql('use experian_raw')
    
    df = spark.sql('''
    SELECT 
    ovind.ovind,
    ovind.luid,
    ex.lngth_of_res as lengthresidence,
    ex.rec_perscnt as personnum_unit,
    ex.echv as estimatedhomevalue,
    ex.mosaic_hh_v3 as mosaichousehold,
    ex.rsiz_bedr as homebedroomcnt,
    ex.bdat_bdat_1 as p1_bod,
    ex.combined_age_1 as p1_combinedage,
    ex.educ_model_1 as p1_education,
    ex.mrtl_model_1 as p1_maritalstatus,
    ex.bdat_bdat_2 as p2_bod,
    ex.combined_age_2 as p2_combinedage,
    ex.educ_model_2 as p2_education,
    ex.mrtl_model_2 as p2_maritalstatus,
    ex.bdat_bdat_3 as p3_bod,
    ex.combined_age_3 as p3_combinedage,
    ex.educ_model_3 as p3_education,
    ex.mrtl_model_3 as p3_maritalstatus,
    ex.bdat_bdat_4 as p4_bod,
    ex.combined_age_4 as p4_combinedage,
    ex.educ_model_4 as p4_education,
    ex.mrtl_model_4 as p4_maritalstatus,
    ex.bdat_bdat_5 as p5_bod,
    ex.combined_age_5 as p5_combinedage,
    ex.educ_model_5 as p5_education,
    ex.mrtl_model_5 as p5_maritalstatus,
    ex.bdat_bdat_6 as p6_bod,
    ex.combined_age_6 as p6_combinedage,
    ex.educ_model_6 as p6_education,
    ex.mrtl_model_6 as p6_maritalstatus,
    ex.quarteryearkey,
    ROW_NUMBER() over (partition by ovind.ovind order by ex.bdat_bdat_1 desc) as rowno
FROM
(
    SELECT distinct ovind, luid
    FROM experian_raw_access.consumerview_ind_starts_q12020_vw
    WHERE quarteryearkey = '{0}'  
) AS ovind 
INNER JOIN
(
    SELECT distinct recd_luid
    , lngth_of_res
    , rec_perscnt
    , echv
    , mosaic_hh_v3
    , rsiz_bedr
    , bdat_bdat_1
    , combined_age_1
    , educ_model_1
    , mrtl_model_1
    , bdat_bdat_2
    , combined_age_2 
    , educ_model_2
    , mrtl_model_2
    , bdat_bdat_3
    , combined_age_3
    , educ_model_3
    , mrtl_model_3
    , bdat_bdat_4
    , combined_age_4
    , educ_model_4
    , mrtl_model_4
    , bdat_bdat_5
    , combined_age_5
    , educ_model_5
    , mrtl_model_5
    , bdat_bdat_6
    , combined_age_6
    , educ_model_6
    , mrtl_model_6
    , quarteryearkey
    FROM experian_raw_access.consumerview_starts_q12020_vw
    WHERE recd_luid <> '0000000000' and quarteryearkey = '{1}' 
) as ex 
ON ovind.luid = ex.recd_luid
    '''.format(match_dt, experian_dt))
    
    return df

In [ ]:
%%spark

df_experian = load_ExperianData_id(match_dt='Q12020', experian_dt='Q12020')
df_experian.count()